In [2]:
# API Dokumentationen:
# https://bmrs.elexon.co.uk/api-documentation/endpoint/forecast/
# https://api.rebase.energy/weather/docs/v2/#general-usage
# https://api.rebase.energy/challenges/redoc#tag/

In [3]:
import comp_utils
rebase_api_client = comp_utils.RebaseAPI(api_key = open("DanMaLeo_key.txt").read())

In [4]:
rebase_api_client

In [5]:
import logging
logging.getLogger().setLevel(logging.INFO)
logging.info("test")

INFO:root:test


In [6]:
myDict = {'a': 1, 'b': 2}
new_data = {'c': 3, 'd': 4}

myDict.update(new_data)

print(myDict)

{'a': 1, 'b': 2, 'c': 3, 'd': 4}


In [7]:
import comp_utils
import pandas as pd

In [8]:
rebase_api_client = comp_utils.RebaseAPI(api_key = open("DanMaLeo_key.txt").read())

In [9]:
# Nationaler Forecast für Solar- und Windproduktion
# timestamp_utc bzw. settlement_date und settlement_period (23:00 (Periode 1)- 22:30 (Periode 48)) sind hier die valid_time
solar_wind_forecast = comp_utils.RebaseAPI.get_solar_wind_forecast(rebase_api_client, "2023-10-11")
solar_wind_forecast.tail()

,timestamp_utc,settlement_date,settlement_period,solar_mw,wind_offshore_mw,wind_onshore_mw
43,2023-10-11T19:30:00Z,2023-10-11,42,0.0,5119.954667,4578.129000
44,2023-10-11T20:00:00Z,2023-10-11,43,0.0,5010.592333,4013.771000
45,2023-10-11T20:30:00Z,2023-10-11,44,0.0,5010.592333,4013.771000
46,2023-10-11T21:00:00Z,2023-10-11,45,0.0,4603.203000,3513.663667
47,2023-10-11T21:30:00Z,2023-10-11,46,0.0,4603.203000,3513.663667


In [10]:
# Forecast für day-ahead-demand
# national_demand: NDF berücksichtigt die Übertragungsverluste, aber nicht die Transformatorenlast, die Pumpspeichernachfrage oder die Interkonnektornachfrage; die Daten werden nur auf nationaler Ebene gemeldet
# transmition_system_demand: TSDF berücksichtigt die Übertragungsverluste, die Transformatorenlast, die Pumpspeichernachfrage und die Interkonnektornachfrage. Die Daten werden sowohl auf nationaler Ebene als auch auf der Ebene der Grenzgebiete (Systemzonen) gemeldet
# boundary: geografischer Bezugsbereich (https://www.arcgis.com/apps/webappviewer/index.html?id=cb3474a78df24139b1651908ff8c8975)
# keine historischen Daten verfügbar
da_demand_forecast = comp_utils.RebaseAPI.get_day_ahead_demand_forecast(rebase_api_client) # Gibt Liste zurück
da_demand_forecast = pd.DataFrame(da_demand_forecast)
da_demand_forecast

<Response [200]>


,timestamp_utc,settlement_date,settlement_period,boundary,publish_time_utc,transmission_system_demand,national_demand
0,2024-10-13T19:00:00Z,2024-10-13,41,N,2024-10-13T18:45:00Z,32884.0,31928.0
1,2024-10-13T19:30:00Z,2024-10-13,42,N,2024-10-13T18:45:00Z,31962.0,30855.0
2,2024-10-13T20:00:00Z,2024-10-13,43,N,2024-10-13T18:45:00Z,30759.0,29677.0
3,2024-10-13T20:30:00Z,2024-10-13,44,N,2024-10-13T18:45:00Z,29252.0,28080.0
4,2024-10-13T21:00:00Z,2024-10-13,45,N,2024-10-13T18:45:00Z,27355.0,26322.0
...,...,...,...,...,...,...,...
61,2024-10-15T01:30:00Z,2024-10-15,6,N,2024-10-13T18:45:00Z,22837.0,21884.0
62,2024-10-15T02:00:00Z,2024-10-15,7,N,2024-10-13T18:45:00Z,22472.0,21520.0
63,2024-10-15T02:30:00Z,2024-10-15,8,N,2024-10-13T18:45:00Z,22617.0,21666.0
64,2024-10-15T03:00:00Z,2024-10-15,9,N,2024-10-13T18:45:00Z,22272.0,21323.0


In [11]:
# Forecast für den Unterschied zwischen dem demand und der verfügbaren Generstions- Kapazität 2 bis 14 Tage im Voraus (nützlich für price forecasting)
# keine historischen Daten verfügbar
# forecast_date: valid_time
margin_forecast = comp_utils.RebaseAPI.get_margin_forecast(rebase_api_client) # Gibt Liste zurück
margin_forecast = pd.DataFrame(margin_forecast)
margin_forecast

<Response [200]>


,forecast_date,publish_time_utc,margin
0,2024-10-15,2024-10-13T18:00:00Z,17741.0
1,2024-10-16,2024-10-13T18:00:00Z,17999.0
2,2024-10-17,2024-10-13T18:00:00Z,9404.0
3,2024-10-18,2024-10-13T18:00:00Z,22108.0
4,2024-10-19,2024-10-13T18:00:00Z,22436.0
5,2024-10-20,2024-10-13T18:00:00Z,23540.0
6,2024-10-21,2024-10-13T18:00:00Z,17421.0
7,2024-10-22,2024-10-13T18:00:00Z,13842.0
8,2024-10-23,2024-10-13T18:00:00Z,11480.0
9,2024-10-24,2024-10-13T18:00:00Z,13090.0


In [12]:
# Aktuelle Wetterforecasts für Hornsea 1 von DWD
hornsea_dwd = comp_utils.RebaseAPI.get_hornsea_dwd(rebase_api_client)
hornsea_dwd.tail()

200


,RelativeHumidity_latitude_53.77_longitude_1.702,RelativeHumidity_latitude_53.77_longitude_1.767,RelativeHumidity_latitude_53.77_longitude_1.832,RelativeHumidity_latitude_53.77_longitude_1.897,RelativeHumidity_latitude_53.77_longitude_1.962,RelativeHumidity_latitude_53.77_longitude_2.027,RelativeHumidity_latitude_53.84_longitude_1.702,RelativeHumidity_latitude_53.84_longitude_1.767,RelativeHumidity_latitude_53.84_longitude_1.832,RelativeHumidity_latitude_53.84_longitude_1.897,...,WindSpeed_latitude_54.03_longitude_1.962,WindSpeed_latitude_54.03_longitude_2.027,WindSpeed_latitude_54.1_longitude_1.702,WindSpeed_latitude_54.1_longitude_1.767,WindSpeed_latitude_54.1_longitude_1.832,WindSpeed_latitude_54.1_longitude_1.897,WindSpeed_latitude_54.1_longitude_1.962,WindSpeed_latitude_54.1_longitude_2.027,ref_datetime,valid_datetime
88,87.259117,86.945374,86.788498,86.474747,86.317871,86.004120,87.415993,87.102249,86.945374,86.631622,...,4.098991,4.046521,3.979132,4.012907,4.047252,4.026607,4.006631,3.969180,"Sun, 13 Oct 2024 12:00:00 GMT","Fri, 18 Oct 2024 00:00:00 GMT"
89,87.102249,87.259117,87.259117,87.415993,87.259117,86.945374,87.572868,87.572868,87.572868,87.572868,...,7.270855,7.270855,6.799082,6.730636,6.774969,6.774969,6.774969,6.808295,"Sun, 13 Oct 2024 12:00:00 GMT","Fri, 18 Oct 2024 03:00:00 GMT"
90,86.945374,86.788498,86.160995,85.533501,85.219749,84.592255,87.572868,87.102249,86.474747,85.847244,...,8.652487,8.298055,8.579968,8.550322,8.603376,8.574615,8.518016,8.310818,"Sun, 13 Oct 2024 12:00:00 GMT","Fri, 18 Oct 2024 06:00:00 GMT"
91,88.670990,88.827866,88.827866,88.984741,88.984741,88.827866,88.514114,88.514114,88.670990,88.984741,...,8.479960,8.630918,8.332643,8.390779,8.428046,8.519752,8.601488,8.740711,"Sun, 13 Oct 2024 12:00:00 GMT","Fri, 18 Oct 2024 09:00:00 GMT"
92,82.396004,82.552879,82.396004,82.552879,82.866631,83.494125,83.023506,83.180382,82.552879,82.396004,...,9.610056,9.641164,10.259303,10.151072,9.970610,9.899035,9.758137,9.818524,"Sun, 13 Oct 2024 12:00:00 GMT","Fri, 18 Oct 2024 12:00:00 GMT"


In [13]:
# Aktuelle Wetterforecasts für Hornsea 1 von gfs (ncep)
hornsea_gfs = comp_utils.RebaseAPI.get_hornsea_gfs(rebase_api_client)
hornsea_gfs.tail()

200


,RelativeHumidity_latitude_53.59_longitude_1.522,RelativeHumidity_latitude_53.59_longitude_1.772,RelativeHumidity_latitude_53.59_longitude_2.022,RelativeHumidity_latitude_53.84_longitude_1.522,RelativeHumidity_latitude_53.84_longitude_1.772,RelativeHumidity_latitude_53.84_longitude_2.022,RelativeHumidity_latitude_54.09_longitude_1.522,RelativeHumidity_latitude_54.09_longitude_1.772,RelativeHumidity_latitude_54.09_longitude_2.022,Temperature_latitude_53.59_longitude_1.522,...,WindSpeed_latitude_53.59_longitude_1.772,WindSpeed_latitude_53.59_longitude_2.022,WindSpeed_latitude_53.84_longitude_1.522,WindSpeed_latitude_53.84_longitude_1.772,WindSpeed_latitude_53.84_longitude_2.022,WindSpeed_latitude_54.09_longitude_1.522,WindSpeed_latitude_54.09_longitude_1.772,WindSpeed_latitude_54.09_longitude_2.022,ref_datetime,valid_datetime
204,73.695808,72.467415,71.889343,72.106117,71.383530,70.733208,72.900963,71.889343,71.383530,10.436615,...,10.998967,11.037196,10.917126,11.048465,11.402158,11.107772,11.329076,11.471389,"Sun, 13 Oct 2024 12:00:00 GMT","Tue, 29 Oct 2024 00:00:00 GMT"
205,76.098404,74.996460,72.900963,73.515160,71.889343,70.877724,73.298386,72.178375,71.383530,10.021667,...,9.713751,9.911674,10.244092,10.222232,10.111067,10.452024,10.236080,10.020371,"Sun, 13 Oct 2024 12:00:00 GMT","Tue, 29 Oct 2024 03:00:00 GMT"
206,74.888077,74.599037,74.093231,76.798409,76.604218,75.791306,75.104851,75.104851,74.815819,9.980164,...,11.352020,11.287741,11.140441,11.161592,11.302924,11.446057,11.497641,11.711603,"Sun, 13 Oct 2024 12:00:00 GMT","Tue, 29 Oct 2024 06:00:00 GMT"
207,79.006813,78.103584,77.200348,78.699715,77.701645,77.100998,78.591331,77.801003,76.897766,10.104645,...,12.419416,12.053014,12.687449,12.419416,12.061338,13.160488,12.925119,12.705350,"Sun, 13 Oct 2024 12:00:00 GMT","Tue, 29 Oct 2024 09:00:00 GMT"
208,81.102318,79.097137,78.202942,81.210701,79.982307,79.982307,82.728134,82.222321,81.897163,10.768555,...,14.449369,14.045424,14.148578,14.122671,14.158862,14.112075,14.037350,14.148578,"Sun, 13 Oct 2024 12:00:00 GMT","Tue, 29 Oct 2024 12:00:00 GMT"


In [14]:
# Aktuelle Wetterforecasts für pes10
pes10_nwp = comp_utils.RebaseAPI.get_pes10_nwp(rebase_api_client,
                                               model="NCEP_GFS")
pes10_nwp

200


,valid_datetime,ref_datetime,SolarDownwardRadiation,CloudCover,Temperature,point,latitude,longitude
0,2024-10-13T12:00:00Z,2024-10-13T12:00:00Z,333.45,0.76,9.40,0,52.487256,0.401245
1,2024-10-13T13:00:00Z,2024-10-13T12:00:00Z,339.48,1.00,9.86,0,52.487256,0.401245
2,2024-10-13T14:00:00Z,2024-10-13T12:00:00Z,168.46,1.00,9.65,0,52.487256,0.401245
3,2024-10-13T15:00:00Z,2024-10-13T12:00:00Z,91.68,1.00,9.44,0,52.487256,0.401245
4,2024-10-13T16:00:00Z,2024-10-13T12:00:00Z,34.88,1.00,8.67,0,52.487256,0.401245
...,...,...,...,...,...,...,...,...
204,2024-10-29T00:00:00Z,2024-10-13T12:00:00Z,0.00,0.10,6.58,19,52.078072,-0.011349
205,2024-10-29T03:00:00Z,2024-10-13T12:00:00Z,0.00,1.00,5.96,19,52.078072,-0.011349
206,2024-10-29T06:00:00Z,2024-10-13T12:00:00Z,0.00,1.00,6.43,19,52.078072,-0.011349
207,2024-10-29T09:00:00Z,2024-10-13T12:00:00Z,133.90,1.00,8.71,19,52.078072,-0.011349


In [15]:
# Aktuelle Wetterforecasts für die Bevölkerung
demand_nwp = comp_utils.RebaseAPI.get_demand_nwp(rebase_api_client,
                                                 model="NCEP_GFS")
demand_nwp

200


,valid_datetime,ref_datetime,Temperature,WindSpeed,WindDirection,TotalPrecipitation,RelativeHumidity,point,latitude,longitude
0,2024-10-13T12:00:00Z,2024-10-13T12:00:00Z,9.90,1.86,332.50,0.00,60.11,0,51.479,-0.451
1,2024-10-13T13:00:00Z,2024-10-13T12:00:00Z,10.23,1.70,343.02,0.00,56.50,0,51.479,-0.451
2,2024-10-13T14:00:00Z,2024-10-13T12:00:00Z,10.31,0.93,344.57,0.00,55.92,0,51.479,-0.451
3,2024-10-13T15:00:00Z,2024-10-13T12:00:00Z,10.15,0.79,48.62,0.00,56.79,0,51.479,-0.451
4,2024-10-13T16:00:00Z,2024-10-13T12:00:00Z,9.98,0.39,90.99,0.00,58.52,0,51.479,-0.451
...,...,...,...,...,...,...,...,...,...,...
204,2024-10-29T00:00:00Z,2024-10-13T12:00:00Z,7.59,5.39,255.84,0.00,77.30,6,54.297,-1.533
205,2024-10-29T03:00:00Z,2024-10-13T12:00:00Z,7.20,4.60,241.35,0.00,79.60,6,54.297,-1.533
206,2024-10-29T06:00:00Z,2024-10-13T12:00:00Z,8.51,6.43,245.96,0.01,77.40,6,54.297,-1.533
207,2024-10-29T09:00:00Z,2024-10-13T12:00:00Z,9.32,8.16,220.30,0.12,82.51,6,54.297,-1.533


In [16]:
# Aktuelle Wetter Forecasts für beliebige Punkte
# forecast_date: valid_time
# keine historischen Daten verfügbar
weather_latest = comp_utils.RebaseAPI.query_weather_latest(rebase_api_client, 
                                                           model="DWD_ICON-EU",
                                                           lats='60.1, 61.2, 59.33',
                                                           lons='17.2, 13.1, 18.05',
                                                           variables='Temperature, WindSpeed, CloudCover',
                                                           query_type="GET") # query_type: GET, POST, ...
weather_latest = pd.DataFrame(margin_forecast)
weather_latest

400


,forecast_date,publish_time_utc,margin
0,2024-10-15,2024-10-13T18:00:00Z,17741.0
1,2024-10-16,2024-10-13T18:00:00Z,17999.0
2,2024-10-17,2024-10-13T18:00:00Z,9404.0
3,2024-10-18,2024-10-13T18:00:00Z,22108.0
4,2024-10-19,2024-10-13T18:00:00Z,22436.0
5,2024-10-20,2024-10-13T18:00:00Z,23540.0
6,2024-10-21,2024-10-13T18:00:00Z,17421.0
7,2024-10-22,2024-10-13T18:00:00Z,13842.0
8,2024-10-23,2024-10-13T18:00:00Z,11480.0
9,2024-10-24,2024-10-13T18:00:00Z,13090.0


In [ ]:
# Funktioniert nicht
weather_latest_points = comp_utils.RebaseAPI.query_weather_latest_points(rebase_api_client,
                                                                         model="DWD_ICON-EU",
                                                                         lats='60.1, 61.2, 59.33',
                                                                         lons='17.2, 13.1, 18.05',
                                                                         variables='Temperature, WindSpeed, CloudCover')

In [ ]:
# Funktioniert nicht
weather_latest_points = comp_utils.RebaseAPI.query_weather_latest_grid(rebase_api_client,
                                                                       model="DWD_ICON-EU",
                                                                       lats='60.1, 61.2, 59.33',
                                                                       lons='17.2, 13.1, 18.05',
                                                                       variables='Temperature, WindSpeed, CloudCover')

In [18]:
# Konvertiert einen Dataframe zu einem Dataset
latest_dwd_Hornsea1 = comp_utils.weather_df_to_xr(rebase_api_client.get_hornsea_dwd())
latest_dwd_Hornsea1

200


<xarray.Dataset> Size: 162kB
Dimensions:            (ref_datetime: 1, valid_datetime: 93, longitude: 6,
                        latitude: 6)
Coordinates:
  * ref_datetime       (ref_datetime) object 8B 1728820800000000000
  * valid_datetime     (valid_datetime) object 744B 1728820800000000000 ... 1...
  * longitude          (longitude) object 48B '1.702' '1.767' ... '2.027'
  * latitude           (latitude) object 48B '53.77' '53.84' ... '54.03' '54.1'
Data variables:
    RelativeHumidity   (ref_datetime, valid_datetime, longitude, latitude) float64 27kB ...
    Temperature        (ref_datetime, valid_datetime, longitude, latitude) float64 27kB ...
    WindDirection      (ref_datetime, valid_datetime, longitude, latitude) float64 27kB ...
    WindDirection:100  (ref_datetime, valid_datetime, longitude, latitude) float64 27kB ...
    WindSpeed          (ref_datetime, valid_datetime, longitude, latitude) float64 27kB ...
    WindSpeed:100      (ref_datetime, valid_datetime, longitude, latitude) float64 27kB ...

In [27]:
# Submit Methoden:
# day_ahead_market_times(today_date=pd.to_datetime('today'))
# comp_utils.prep_submission_in_json_format(weather_data)
# comp_utils.RebaseAPI.submit(rebase_api_client, data)

### AutoSubmitter Example

In [1]:
from model_utils import *
from Preprocessing import FeatureEngineerer

merged_hornsea = pd.read_parquet("preprocessed_hornsea_with_energy.parquet")
merged_pes = pd.read_parquet("preprocessed_pes_with_energy.parquet")

feature_engineerer_wind = FeatureEngineerer(label = 'Wind_MWh_credit', columns_to_ohe=['unavailabilityType', 'affectedUnit', 'outage'])
feature_engineerer_wind.perform_feature_engineering(merged_hornsea, deployment = False)

feature_engineerer_solar = FeatureEngineerer(label = "Solar_MWh_credit", columns_to_ohe=[])
feature_engineerer_solar.perform_feature_engineering(merged_pes, deployment = False)

wind_model = LGBMRegressorModel(feature_engineerer_wind, quantiles=np.round(np.arange(0.1, 1.0, 0.1), 1), model_save_dir="lgbm_model_wind", load_pretrained=True)
solar_model = LGBMRegressorModel(feature_engineerer_solar, quantiles=np.round(np.arange(0.1, 1.0, 0.1), 1), model_save_dir="lgbm_model_solar", load_pretrained=True)

from submissions import AutoSubmitter

submitter = AutoSubmitter(wind_model, solar_model)
submitter.fetch_data().prepare_data().predict().prepare_submission()

Loaded pretrained Quantile Regressor model for quantile 0.1 from lgbm_model_wind\lgbm_model_quantile_0.1.pkl
Loaded pretrained Quantile Regressor model for quantile 0.2 from lgbm_model_wind\lgbm_model_quantile_0.2.pkl
Loaded pretrained Quantile Regressor model for quantile 0.3 from lgbm_model_wind\lgbm_model_quantile_0.3.pkl
Loaded pretrained Quantile Regressor model for quantile 0.4 from lgbm_model_wind\lgbm_model_quantile_0.4.pkl
Loaded pretrained Quantile Regressor model for quantile 0.5 from lgbm_model_wind\lgbm_model_quantile_0.5.pkl
Loaded pretrained Quantile Regressor model for quantile 0.6 from lgbm_model_wind\lgbm_model_quantile_0.6.pkl
Loaded pretrained Quantile Regressor model for quantile 0.7 from lgbm_model_wind\lgbm_model_quantile_0.7.pkl
Loaded pretrained Quantile Regressor model for quantile 0.8 from lgbm_model_wind\lgbm_model_quantile_0.8.pkl
Loaded pretrained Quantile Regressor model for quantile 0.9 from lgbm_model_wind\lgbm_model_quantile_0.9.pkl
Loaded pretrained Q

INFO:root:Perform data cleaning on the weather data...
INFO:root:Merge weather stations...
c:\Users\Leonard\vscode\awp2\Preprocessing.py:658: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  weather_data = weather_data.resample("30min", level = 1).mean().interpolate("time")
INFO:root:Merge with outages data (REMIT)...
c:\Users\Leonard\vscode\awp2\Preprocessing.py:326: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  outages_df = pd.concat([outages_df, json_data_with_date_ranges])
c:\Users\Leonard\vscode\awp2\Preprocessing.py:326: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this

                                  q10         q20         q30         q40  \
val_time                                                                    
2024-10-13 17:00:00+00:00  292.379460  228.349356  166.086423  225.992814   
2024-10-13 17:30:00+00:00  325.103879  227.204664  173.677517  221.820978   
2024-10-13 18:00:00+00:00  322.698444  227.204664  173.612649  222.269203   
2024-10-13 18:30:00+00:00  321.715530  227.204664  169.396911  225.284859   
2024-10-13 19:00:00+00:00  321.715530  227.204664  175.692598  243.590413   
2024-10-13 19:30:00+00:00  320.263439  236.526998  211.169232  250.392353   
2024-10-13 20:00:00+00:00  308.733919  210.064724  203.013605  247.402487   
2024-10-13 20:30:00+00:00  320.853583  225.919023  204.702680  243.439836   
2024-10-13 21:00:00+00:00  322.668874  225.991526  194.979482  243.439836   
2024-10-13 21:30:00+00:00  195.764113  200.852114  150.005376  233.262207   
2024-10-13 22:00:00+00:00  108.499823   53.111570  110.444837   95.793735   

In [3]:
submitter.predictions["final"]

{'market_day': '2024-10-14',
 'submission': [{'timestamp': '2024-10-13T23:00:00+01:00',
   'market_bid': 133.4410858608776,
   'probabilistic_forecast': {10: 108.49982330486318,
    20: 53.1115702512763,
    30: 110.44483713609036,
    40: 95.79373480064353,
    50: 133.4410858608776,
    60: 151.70192207669865,
    70: 188.42289613746632,
    80: 219.63923478885647,
    90: 236.64362054857978}},
  {'timestamp': '2024-10-13T23:30:00+01:00',
   'market_bid': 109.5154966281777,
   'probabilistic_forecast': {10: 97.86512523753387,
    20: 35.50699261165486,
    30: 84.42550155488611,
    40: 87.48303397378864,
    50: 109.5154966281777,
    60: 132.77258499427313,
    70: 153.6863277701298,
    80: 180.29985787273588,
    90: 253.01603351727778}},
  {'timestamp': '2024-10-14T00:00:00+01:00',
   'market_bid': 94.81288834737667,
   'probabilistic_forecast': {10: 95.80123126637979,
    20: 32.345073960734695,
    30: 73.36586702956059,
    40: 75.60627542278175,
    50: 94.81288834737667,
  